In [3]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageFilter
import matplotlib.cm as cm
import cv2
from __future__ import print_function
from pathlib import Path
import shutil


# Define path variables

In [5]:
dataset_path = Path("/mnt/d/ShapeNetCore.v2/ShapeNetCore.v2")
current_path = Path("./")
image_path = current_path / "dataset"
image_path.mkdir(parents=True, exist_ok=True)
data_path = current_path / "outline"
data_path.mkdir(parents=True, exist_ok=True)
sample_path =  current_path / "sampled"
sample_path.mkdir(parents=True, exist_ok=True)
mask_path_highres = current_path / "mask_highres"
mask_path_highres.mkdir(parents=True, exist_ok=True)
mask_path = current_path / "mask"
mask_path.mkdir(parents=True, exist_ok=True)

# Gather 2D images from ShapeNet base folder

In [23]:
for img in dataset_path.rglob("screenshots/*-6.png"):
    image_name = img.name
    obj_cls = img.parent.parent.parent.name
    if obj_cls != "02691156":
        continue
    obj_id = img.parent.parent.name
    print(f"cls : {obj_cls} - id : {obj_id} - image_name : {image_name}")
    img_folder = (image_path / obj_cls)
    img_folder.mkdir(parents=True, exist_ok=True)
    shutil.copy(img, img_folder / image_name)

cls : 02691156 - id : 10155655850468db78d106ce0a280f87 - image_name : 10155655850468db78d106ce0a280f87-6.png
cls : 02691156 - id : 1066b65c30d153e04c3a35cee92bb95b - image_name : 1066b65c30d153e04c3a35cee92bb95b-6.png
cls : 02691156 - id : 10e0a7255d279a419751c7a6f15617f4 - image_name : 10e0a7255d279a419751c7a6f15617f4-6.png
cls : 02691156 - id : 11d2af04fad0a7e2ce19d55bc5e6a406 - image_name : 11d2af04fad0a7e2ce19d55bc5e6a406-6.png
cls : 02691156 - id : 11def84f9fe8ab0acd726e804e5d7a - image_name : 11def84f9fe8ab0acd726e804e5d7a-6.png
cls : 02691156 - id : 122776d17b6a118086da73d36506db6f - image_name : 122776d17b6a118086da73d36506db6f-6.png
cls : 02691156 - id : 122963149f6a04272620819ddac40644 - image_name : 122963149f6a04272620819ddac40644-6.png
cls : 02691156 - id : 123ac29b0aac8c8e5d07633dda45110 - image_name : 123ac29b0aac8c8e5d07633dda45110-6.png
cls : 02691156 - id : 125417c3fa9bc1b130f57e42c77d3894 - image_name : 125417c3fa9bc1b130f57e42c77d3894-6.png
cls : 02691156 - id : 128

cls : 02691156 - id : 1e2de00cf19a0a33554ccf8c30febe7 - image_name : 1e2de00cf19a0a33554ccf8c30febe7-6.png
cls : 02691156 - id : 1e358e70c047687a1a8831797284b67 - image_name : 1e358e70c047687a1a8831797284b67-6.png
cls : 02691156 - id : 1e44b99c8de5eb01ebc54ed98d6399b2 - image_name : 1e44b99c8de5eb01ebc54ed98d6399b2-6.png
cls : 02691156 - id : 1e6a71e0cb436a88a3a1394d6e3d2c63 - image_name : 1e6a71e0cb436a88a3a1394d6e3d2c63-6.png
cls : 02691156 - id : 1e8adc1e176bc68571d03b466c72ce41 - image_name : 1e8adc1e176bc68571d03b466c72ce41-6.png
cls : 02691156 - id : 1ed9876ed48c145d663e90eaf6b4ca52 - image_name : 1ed9876ed48c145d663e90eaf6b4ca52-6.png
cls : 02691156 - id : 1f08b579e153b2de313f9af5275b7c70 - image_name : 1f08b579e153b2de313f9af5275b7c70-6.png
cls : 02691156 - id : 1f672d2fd5e3f4e78026abe712c1ab05 - image_name : 1f672d2fd5e3f4e78026abe712c1ab05-6.png
cls : 02691156 - id : 1f7a516fdf80dcdc1006ed55bc1a3fc - image_name : 1f7a516fdf80dcdc1006ed55bc1a3fc-6.png
cls : 02691156 - id : 201

cls : 02691156 - id : 2c1fff0653854166e7a636089598229 - image_name : 2c1fff0653854166e7a636089598229-6.png
cls : 02691156 - id : 2c49289098e4492bca8607f540cc62ba - image_name : 2c49289098e4492bca8607f540cc62ba-6.png
cls : 02691156 - id : 2d0945065aa2a427eec7473defebb526 - image_name : 2d0945065aa2a427eec7473defebb526-6.png
cls : 02691156 - id : 2d255ec7a05b301a203936772104a82d - image_name : 2d255ec7a05b301a203936772104a82d-6.png
cls : 02691156 - id : 2d43c1430df8194ace5721ccacba16 - image_name : 2d43c1430df8194ace5721ccacba16-6.png
cls : 02691156 - id : 2d4a57a467307d675e9e2656aff7dd5b - image_name : 2d4a57a467307d675e9e2656aff7dd5b-6.png
cls : 02691156 - id : 2d4c147d4d1cc40c582bf3113c3974ef - image_name : 2d4c147d4d1cc40c582bf3113c3974ef-6.png
cls : 02691156 - id : 2d7562f5bf2c7f2da1d85548168d6015 - image_name : 2d7562f5bf2c7f2da1d85548168d6015-6.png
cls : 02691156 - id : 2d9a7863dcfb5663d208f79049825a82 - image_name : 2d9a7863dcfb5663d208f79049825a82-6.png
cls : 02691156 - id : 2e4

cls : 02691156 - id : 3aa98ec8f8a1c1f44d210d9468aedaf2 - image_name : 3aa98ec8f8a1c1f44d210d9468aedaf2-6.png
cls : 02691156 - id : 3ab1e94b6c3a1730c56cc5a87f567365 - image_name : 3ab1e94b6c3a1730c56cc5a87f567365-6.png
cls : 02691156 - id : 3ae96a1e1bb488942296d88107d065f6 - image_name : 3ae96a1e1bb488942296d88107d065f6-6.png
cls : 02691156 - id : 3b3b2777be8b6b31ca8607f540cc62ba - image_name : 3b3b2777be8b6b31ca8607f540cc62ba-6.png
cls : 02691156 - id : 3b3eb57af5f995c268aa0dbe2db661e2 - image_name : 3b3eb57af5f995c268aa0dbe2db661e2-6.png
cls : 02691156 - id : 3b529633bdd49dc1e5dbe91fa3801c4c - image_name : 3b529633bdd49dc1e5dbe91fa3801c4c-6.png
cls : 02691156 - id : 3badd7765618bd66a532a2f6f060af39 - image_name : 3badd7765618bd66a532a2f6f060af39-6.png
cls : 02691156 - id : 3bff0d84311470a357f53fe209e96fca - image_name : 3bff0d84311470a357f53fe209e96fca-6.png
cls : 02691156 - id : 3c0a871ed33a1c6a296ee999ff54cc3b - image_name : 3c0a871ed33a1c6a296ee999ff54cc3b-6.png
cls : 02691156 - id

cls : 02691156 - id : 48e477d5904bb7bb1ad94eee1d03defc - image_name : 48e477d5904bb7bb1ad94eee1d03defc-6.png
cls : 02691156 - id : 48fed12884533a5fe48f6bbab67fa514 - image_name : 48fed12884533a5fe48f6bbab67fa514-6.png
cls : 02691156 - id : 49131e925ef3349566cf1b4a8fc3914e - image_name : 49131e925ef3349566cf1b4a8fc3914e-6.png
cls : 02691156 - id : 4982bea0a007c19593b2f224b3acb952 - image_name : 4982bea0a007c19593b2f224b3acb952-6.png
cls : 02691156 - id : 4992d4c425b3cb426e8204b21fc3574b - image_name : 4992d4c425b3cb426e8204b21fc3574b-6.png
cls : 02691156 - id : 49aa625393dbd1a4d443895deb2f03aa - image_name : 49aa625393dbd1a4d443895deb2f03aa-6.png
cls : 02691156 - id : 49e21b81565a51e6493c5f51f14398c7 - image_name : 49e21b81565a51e6493c5f51f14398c7-6.png
cls : 02691156 - id : 49f400e6574c75554c3a35cee92bb95b - image_name : 49f400e6574c75554c3a35cee92bb95b-6.png
cls : 02691156 - id : 4a199b1c3c80c8823bfdf036d38c839b - image_name : 4a199b1c3c80c8823bfdf036d38c839b-6.png
cls : 02691156 - id

...

cls : 02691156 - id : c878cac87819f01490baeef8ba5b93e5 - image_name : c878cac87819f01490baeef8ba5b93e5-6.png
cls : 02691156 - id : c93d0663035962b01b1cc145b0144991 - image_name : c93d0663035962b01b1cc145b0144991-6.png
cls : 02691156 - id : c93f76888cf5a14c93421b5e4ff654e0 - image_name : c93f76888cf5a14c93421b5e4ff654e0-6.png
cls : 02691156 - id : c9584d90a1da19f723a665a253ac8cae - image_name : c9584d90a1da19f723a665a253ac8cae-6.png
cls : 02691156 - id : c9764fd84080da3f4fb6842b3610149 - image_name : c9764fd84080da3f4fb6842b3610149-6.png
cls : 02691156 - id : c9be9f07f5ae7c375d7629390efe0a2 - image_name : c9be9f07f5ae7c375d7629390efe0a2-6.png
cls : 02691156 - id : ca2a07b005ad4385ca8607f540cc62ba - image_name : ca2a07b005ad4385ca8607f540cc62ba-6.png
cls : 02691156 - id : ca359fd1ed701d06f119374ee5d5f944 - image_name : ca359fd1ed701d06f119374ee5d5f944-6.png
cls : 02691156 - id : ca4443e9a37c4f0b16ade7bb280a6832 - image_name : ca4443e9a37c4f0b16ade7bb280a6832-6.png
cls : 02691156 - id : c

cls : 02691156 - id : d6edd8b0801a02baf7571913c73c0c78 - image_name : d6edd8b0801a02baf7571913c73c0c78-6.png
cls : 02691156 - id : d7287ce98d521db5ca8607f540cc62ba - image_name : d7287ce98d521db5ca8607f540cc62ba-6.png
cls : 02691156 - id : d72a483cf8a0cf2bbbf3143b1cb6076a - image_name : d72a483cf8a0cf2bbbf3143b1cb6076a-6.png
cls : 02691156 - id : d7454b0e80d3236ff268234accd8cf4e - image_name : d7454b0e80d3236ff268234accd8cf4e-6.png
cls : 02691156 - id : d78a16856adad344670aaa01f77ae41a - image_name : d78a16856adad344670aaa01f77ae41a-6.png
cls : 02691156 - id : d7e8b636d4a7dab67e697b0700f10b81 - image_name : d7e8b636d4a7dab67e697b0700f10b81-6.png
cls : 02691156 - id : d7f3d0503ee4ee1cc34b900bb2492e - image_name : d7f3d0503ee4ee1cc34b900bb2492e-6.png
cls : 02691156 - id : d8719b491412ea0bb1cb87d703a74b6f - image_name : d8719b491412ea0bb1cb87d703a74b6f-6.png
cls : 02691156 - id : d8a43017132c210cc1006ed55bc1a3fc - image_name : d8a43017132c210cc1006ed55bc1a3fc-6.png
cls : 02691156 - id : d

cls : 02691156 - id : e3f562bd552fbb14496e2f80fb255499 - image_name : e3f562bd552fbb14496e2f80fb255499-6.png
cls : 02691156 - id : e3fd510add7b1aa3c19eb6ab3736de88 - image_name : e3fd510add7b1aa3c19eb6ab3736de88-6.png
cls : 02691156 - id : e431f79ac9f0266bca677733d59db4df - image_name : e431f79ac9f0266bca677733d59db4df-6.png
cls : 02691156 - id : e4665d76bf8fc441536d5be52cb9d26a - image_name : e4665d76bf8fc441536d5be52cb9d26a-6.png
cls : 02691156 - id : e4e98f8654d29536dc858dada15498d2 - image_name : e4e98f8654d29536dc858dada15498d2-6.png
cls : 02691156 - id : e521828113e1e0c45e28caa3b26a73fd - image_name : e521828113e1e0c45e28caa3b26a73fd-6.png
cls : 02691156 - id : e533a1a758a1752b187ea8c086d9f2c - image_name : e533a1a758a1752b187ea8c086d9f2c-6.png
cls : 02691156 - id : e59c4f290d8585a862b600da24e0965 - image_name : e59c4f290d8585a862b600da24e0965-6.png
cls : 02691156 - id : e5a7a353d5fa8df844b2fa2cac0778f5 - image_name : e5a7a353d5fa8df844b2fa2cac0778f5-6.png
cls : 02691156 - id : e

cls : 02691156 - id : f3e2df468c15795872517bb0a6b4d3ef - image_name : f3e2df468c15795872517bb0a6b4d3ef-6.png
cls : 02691156 - id : f405858991b190173af8c56bd4352c9 - image_name : f405858991b190173af8c56bd4352c9-6.png
cls : 02691156 - id : f44c0e1e55a3469494f3355d9c061b5a - image_name : f44c0e1e55a3469494f3355d9c061b5a-6.png
cls : 02691156 - id : f46df635ac669a9818b14db3b83de9ff - image_name : f46df635ac669a9818b14db3b83de9ff-6.png
cls : 02691156 - id : f4818cd7c0701fbe752556d244bfb42b - image_name : f4818cd7c0701fbe752556d244bfb42b-6.png
cls : 02691156 - id : f57caba2eed21f2998459325335edae9 - image_name : f57caba2eed21f2998459325335edae9-6.png
cls : 02691156 - id : f58a26c915e7a1dceae7a0fa074b4a2a - image_name : f58a26c915e7a1dceae7a0fa074b4a2a-6.png
cls : 02691156 - id : f58b4ed02fc2fd1d4e7648015fd29a1c - image_name : f58b4ed02fc2fd1d4e7648015fd29a1c-6.png
cls : 02691156 - id : f5a667f5796b9a0b9a6e43b878d5b335 - image_name : f5a667f5796b9a0b9a6e43b878d5b335-6.png
cls : 02691156 - id :

KeyboardInterrupt: 

# Create object masks

In [24]:
for path in image_path.rglob("*-6.png"):
    # Read image
    im_in = cv2.imread(str(path), cv2.IMREAD_GRAYSCALE)

    # Threshold.
    th, im_th = cv2.threshold(im_in, 254, 255, cv2.THRESH_BINARY)
    im_th_inv=cv2.bitwise_not(im_th)

    # # Copy the thresholded image.
    im_floodfill = im_th.copy()

    # # Mask used to flood filling.
    # # Notice the size needs to be 2 pixels than the image.
    h, w = im_th.shape[:2]
    mask = np.zeros((h+2, w+2), np.uint8)

    # # Floodfill from point (0, 0)
    cv2.floodFill(im_floodfill, mask, (0,0), 0)
    # # Combine the two images to get the foreground.
    im_out = im_th_inv | im_floodfill

    save_dir = mask_path_highres / path.parent.name
    save_dir.mkdir(parents=True, exist_ok=True)
    
    cv2.imwrite(str(save_dir / path.name), im_out)

# Resize and crop to object

In [25]:
resize = (100,100)
antialias_threshold = 30

In [26]:
for path in mask_path_highres.rglob("*.png"):
    mask = np.array(Image.open(path))
    
    width , height  = mask.shape
    
    left = np.argmax(np.argmax(mask, axis=0) > 0)
    top = np.argmax(np.argmax(mask, axis=1) > 0)
    right = width - np.argmax(np.argmax(np.flip(mask), axis=0) > 0)
    bottom = height - np.argmax(np.argmax(np.flip(mask), axis=1) > 0)
    
    height, width = bottom - top, right - left
    side = max(height, width)
        
    mask = mask[top:top+height, left:left+width]
    mask = np.pad(mask, (((side-height) // 2, (side-height) // 2), 
                         ((side-width) // 2, (side-width) // 2)))
    
    mask = Image.fromarray(mask).resize(resize, 3)
    mask = np.array(mask, dtype=np.uint8) > antialias_threshold
        
    mask = Image.fromarray(mask)
        
    save_dir = mask_path / path.parent.name
    save_dir.mkdir(parents=True, exist_ok=True)
    

    mask.save(save_dir / path.name)

# Get the outline from masks

In [27]:
for path in mask_path.rglob("*.png"):
    img = Image.open(path).convert("L")
    img = img.filter(ImageFilter.FIND_EDGES)
    
    save_dir = data_path / path.parent.name
    save_dir.mkdir(parents=True, exist_ok=True)
    
    img.save(save_dir / path.name)

# Sample outline points for the input representation

In [28]:
n_sample = (1, 15)
n_from_each = 10

In [29]:
import scipy, threading

In [30]:
np.random.seed(0)

In [32]:
for path in data_path.rglob("*.png"):
    img = Image.open(path)
    img = np.array(img, dtype=np.uint8)
    
    n = np.count_nonzero(img)

    sparse_img = scipy.sparse.coo_matrix(np.array(img))
    
    def get_sample(sparse_img, n, i):
        
        n_sample_ = np.random.randint(*n_sample, size=(1,))[0]
        
        samples = np.random.choice(np.arange(0, n), n_sample_, False)

        sparse_img = scipy.sparse.coo_matrix((sparse_img.data[samples], 
                                          (sparse_img.row[samples], sparse_img.col[samples])), 
                                         (100, 100))
        img = sparse_img.todense()
        img = Image.fromarray(img)


        save_dir = sample_path / path.parent.name
        save_dir.mkdir(parents=True, exist_ok=True)
        img.save(save_dir / (f"{i}_{n_sample_}_" + path.name))
    
    threads = []
    for i in range(n_from_each):
        t = threading.Thread(target=get_sample, args=(sparse_img, n, i))
        threads.append(t)
        t.start()
    for t in threads:
        t.join()